In [1]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import multiprocessing 
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


C:\Users\Carla\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [131]:
#¿Las inversiones deben ser reportadas en el anexo es-4?
print('Ingrese la pregunta:')
pregunta = input()

Ingrese la pregunta:
¿La rentabilidad de las inversiones y la tasa implícita de obligaciones deberá ser calculada en la misma moneda y plazo?


In [132]:
cadpun = string.punctuation + '¿�'
stops = stopwords.words('spanish')


pregunta = pregunta.lower()
pregunta = pregunta.translate(str.maketrans('', '', cadpun))
palabras = pregunta.split()
resultwords = [palabra for palabra in palabras if palabra.lower() not in stops]
preguntaclean = (' '.join(resultwords)).split()
preguntaclean

['rentabilidad',
 'inversiones',
 'tasa',
 'implícita',
 'obligaciones',
 'deberá',
 'ser',
 'calculada',
 'misma',
 'moneda',
 'plazo']

In [133]:
tagged_question = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(pregunta)]
palabras = pregunta.split()

In [5]:
model = gensim.models.Word2Vec.load("./doc2vec_wikipedia_es_pvdbow")
model.docvecs.vectors_docs.shape


(1301830, 300)

In [134]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)

bdtesis = client.BDTesis
preguntas = bdtesis.PreguntasSBS


In [135]:
vector = model.infer_vector(preguntaclean).reshape(1,-1)
vectores = []
preguntas = []
for x in bdtesis.PreguntasSBS.find():
    vectores.append(x['vector'])
for x in bdtesis.PreguntasSBS.find():
    preguntas.append(x['Pregunta'])

In [136]:
#comparación la pregunta ingresada con la BD
proximidades = []
maxprox = 0
cont = 0
similares = {}

for x in vectores:
    vectorpregfrec = np.asarray(x).reshape(1,-1)
    res = cosine_similarity(vectorpregfrec,vector) 
    #similares[float(res[0])] = preguntas[cont]
    similares[preguntas[cont]] = float(res[0]) 
    if (res> maxprox):
        preguntaprox = preguntas[cont]
        maxprox = res
        contprox = cont
    proximidades.append(res)
    cont = cont + 1
preguntaprox


'¿La rentabilidad de las inversiones y la tasa implícita de obligaciones deberá ser calculada en la misma moneda y plazo?: '

In [137]:
max(proximidades)

array([[0.85037246]])

In [138]:
ordenados = sorted(similares.items(), key=lambda kv: kv[1], reverse=True)


In [142]:
print(ordenados[0])
print(ordenados[1])
print(ordenados[2])
print(ordenados[3])
print(ordenados[4])


('¿La rentabilidad de las inversiones y la tasa implícita de obligaciones deberá ser calculada en la misma moneda y plazo?: ', 0.8503724640953783)
('¿El monto a registrar en instrumentos de cuentas por pagar deberá ser negativo? ', 0.7241595258538682)
('¿El importe de la operación será el valor Neto (Deducido de depreciación) o Valor de compra? ¿Deberá ser en su moneda original o en la moneda que se tiene contabilizada?', 0.7079747111206574)
('Para los productos de riesgo y ahorro que garantizan una tasa ¿cuál sería la tasa implícita?', 0.7003280123868492)
('¿Qué quiere decir “código identificador de la base de la tasa variable empleado para el cálculo de los intereses del instrumento de inversión”?', 0.6944714762632589)
